### Step 1: Set up the workspace

In [15]:
import json
from azureml.core import Workspace
from azureml.core.model import Model

#GETTING MY ID:
id = open('id_CONFIDENTIAL.json', 'r')
mi = json.load(id)
my_id = mi["my_id"],

ws = Workspace.create(name="Clase2",
                      subscription_id = mi["my_id"],
                      resource_group = "__hw1__",
                      location = "centralindia")

mname = "model"
registered_model = Model.register(model_path="random_forest_model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Deploying AppInsights with name clase2insights95e6d26412.
Deployed AppInsights with name clase2insights95e6d26412. Took 4.15 seconds.
Deploying KeyVault with name clase2keyvault5e8bbe67c2.
Deploying StorageAccount with name clase2storagee09355fd234.
Deployed KeyVault with name clase2keyvault5e8bbe67c2. Took 22.54 seconds.
Deploying Workspace with name Clase2.
Deployed StorageAccount with name clase2storagee09355fd234. Took 26.21 seconds.
Deployed Workspace with name Clase2. Took 40.66 seconds.
Registering model model


In [16]:
################ score.py ###################
scorepy = f"""
import json
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)

def run(raw_data):
    try: ## Try la predicción.
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)
        
        data_dummies = data.drop(selected_columns.json, axis=1)

        result = model.predict(data_dummies).tolist()
        return json.dumps(result)
        except Exception as e:
            return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [17]:
scorepy = f"""import json
import pandas as pd
import joblib
from azureml.core.model import Model

def init():
    global model
    global selected_columns  # Declare selected_columns as global

    # Load the model
    model_path = Model.get_model_path('{mname}')
    model = joblib.load(model_path)

    # Load the selected columns from the JSON file
    selected_columns_path = Model.get_model_path('selected_columns.json')  # Ensure the JSON file is registered
    with open(selected_columns_path, 'r') as f:
        selected_columns = json.load(f)

def run(raw_data):
    try:
        # Parse input data
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)

        # Drop selected columns
        data_dummies = data.drop(selected_columns, axis=1)

        # Perform prediction
        result = model.predict(data_dummies).tolist()
        return json.dumps(result)
    except Exception as e:
        return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

### Step 2: Inference configuration

In [18]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','joblib','azureml.core.model','json']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\rodri\AppData\Local\Temp\ipykernel_3428\3671233594.py:16: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()